In [ ]:
!pip install earthengine-api geemap

In [ ]:
import ee
import geemap

In [ ]:
# Initialize the Earth Engine module.
ee.Initialize()

In [ ]:
!earthengine authenticate

In [ ]:
# Define the administrative boundary of Pará, Brazil
Admin = ee.FeatureCollection("FAO/GAUL/2015/level1")  # Load the GAUL dataset
para_boundary = Admin.filter(ee.Filter.And(
    ee.Filter.eq('ADM1_NAME', 'Para'),
    ee.Filter.eq('ADM0_NAME', 'Brazil')
))

Create a grid (10x10 = 100 tiles in the example) as needed to extract points...

In [ ]:
# Define the number of tiles in x and y directions
num_tiles_x = 10  # 10 tiles along the x-axis
num_tiles_y = 10  # 10 tiles along the y-axis

In [ ]:
# Calculate the dimensions of each tile
lon_start, lon_end = -59.15588775272494, -45.88440337772494
lat_start, lat_end = -10.054019907719326, 2.812773900498235

lon_edge = (lon_end - lon_start) / num_tiles_x
lat_edge = (lat_end - lat_start) / num_tiles_y

# Create a grid of tiles
polys = []
cell_id = 0
for lon in range(num_tiles_x):
    x1 = lon_start + lon * lon_edge
    x2 = x1 + lon_edge
    for lat in range(num_tiles_y):
        cell_id += 1
        y1 = lat_start + lat * lat_edge
        y2 = y1 + lat_edge
        
        polys.append(ee.Feature(ee.Geometry.Rectangle(x1, y1, x2, y2), {'label': cell_id}))

# Create a FeatureCollection from the list of polygons
grid = ee.FeatureCollection(polys)

In [ ]:
# Optionally, add the tiles as a layer to the map for visualization
Map = geemap.Map()
Map.centerObject(grid, 5)
Map.addLayer(para_boundary, {}, "Para Boundary")
Map.addLayer(grid, {}, 'Tiles')
Map

Set some functions to filter the data and extract points...

In [ ]:
# Define the function to apply quality masks
def apply_quality_mask(image):
    return image.updateMask(image.select('quality_flag').eq(1)).updateMask(image.select('degrade_flag').eq(0))

# Define the function to clip images to the tile geometry
def clip_image_to_tile(image, tile_geometry):
    return image.clip(tile_geometry)

# Define the function to convert images to points and filter out points with no data
def convert_images_to_points(image):
    geometry = image.geometry()
    points = image.sample(region=geometry, scale=30, numPixels=1e9)
    points = points.filter(ee.Filter.notNull(['rh98']))  # Replace 'rh98' with the correct band name if needed
    def add_coordinates(feature):
        coords = feature.geometry().coordinates()
        return feature.set({'latitude': coords.get(1), 'longitude': coords.get(0)})
    points = points.map(add_coordinates)
    return points

The following function works on each tile (one by one) and extract all valid GEDI L2A points converting all raster bands into attribute of the point. It also adds latitude and longitude of the points...

In [ ]:
def process_tile(tile):
    tile_geometry = ee.Feature(tile).geometry()

    # Get GEDI L2A raster collection
    gediL2Araster = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")

    # Apply quality masks
    gedi_l2a_quality_masked = gediL2Araster.map(apply_quality_mask)

    # Clip images to the tile geometry
    gedi_l2a_clipped = gedi_l2a_quality_masked.map(lambda img: clip_image_to_tile(img, tile_geometry))

    # Convert images to points
    gedi_points_tile = gedi_l2a_clipped.map(convert_images_to_points).flatten()

    # Create a valid description and filename prefix
    tile_label = tile.get('label').getInfo()  # Extract the label value
    description = f"GEDI_L2A_tile_{tile_label}"
    filename_prefix = f"tile_{tile_label}"

    # Export the points of this tile to a CSV file
    task = ee.batch.Export.table.toDrive(
        collection=gedi_points_tile,
        description=description,
        folder='GEDI_L2A_Para',  # Specify your folder name here
        fileNamePrefix=filename_prefix,  # Add a prefix for the file name
        fileFormat='CSV'
    )
    task.start()
    print(f'Started export for tile {tile_label}')



Create a list of tiles from the grid created above and run the process_tile function on each tile...

In [ ]:
# Get the list of tiles from the grid
tiles_list = grid.getInfo()['features']

# Process each tile
for tile in tiles_list:
    process_tile(ee.Feature(tile['geometry']).set('label', tile['properties']['label']))


The following is not needed! It is only to extract points for missing or failed tiles!

In [ ]:
def process_tile2(tile):
    tile_geometry = ee.Feature(tile).geometry()

    # Get GEDI L2A raster collection
    gediL2Araster = ee.ImageCollection("LARSE/GEDI/GEDI02_A_002_MONTHLY")

    # Apply quality masks
    gedi_l2a_quality_masked = gediL2Araster.map(apply_quality_mask)

    # Clip images to the tile geometry
    gedi_l2a_clipped = gedi_l2a_quality_masked.map(lambda img: clip_image_to_tile(img, tile_geometry))

    # Convert images to points
    gedi_points_tile = gedi_l2a_clipped.map(convert_images_to_points).flatten()

    # Split the points into two parts
    total_points = gedi_points_tile.size().getInfo()
    half_points = total_points // 2

    part_1 = gedi_points_tile.limit(half_points)
    part_2 = gedi_points_tile.skip(half_points)

    # Create a valid description and filename prefix
    tile_label = tile.get('label').getInfo()  # Extract the label value

    # Export the first part of the points of this tile to a CSV file
    task1 = ee.batch.Export.table.toDrive(
        collection=part_1,
        description=f"GEDI_L2A_tile_{tile_label}_part_1",
        folder='GEDI_L2A_Para',  # Specify your folder name here
        fileNamePrefix=f"tile_{tile_label}_part_1",  # Add a prefix for the file name
        fileFormat='CSV'
    )
    task1.start()
    print(f'Started export for tile {tile_label} part 1')

    # Export the second part of the points of this tile to a CSV file
    task2 = ee.batch.Export.table.toDrive(
        collection=part_2,
        description=f"GEDI_L2A_tile_{tile_label}_part_2",
        folder='GEDI_L2A_Para',  # Specify your folder name here
        fileNamePrefix=f"tile_{tile_label}_part_2",  # Add a prefix for the file name
        fileFormat='CSV'
    )
    task2.start()
    print(f'Started export for tile {tile_label} part 2')


In [ ]:
# Get the list of tiles from the grid
tiles_list = grid.getInfo()['features']

# List of specific tiles to process
specific_tiles = [11, 17, 22, 33, 40, 73]

# Process only specific tiles
for tile in tiles_list:
    tile_label = tile['properties']['label']
    if tile_label in specific_tiles:
        process_tile2(ee.Feature(tile['geometry']).set('label', tile_label))